In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")

online_llm = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

In [2]:
#导入建立 agent 的相关环境
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool #定义工具信息的
from pydantic import BaseModel,Field
from langchain.tools import BaseTool
from typing import Type
import requests

In [3]:
## 定义记忆模块
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_41751/2833621595.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)


In [4]:
# 聚合网相关api调用-根据城市查询天气 - 代码参考（根据实际业务情况修改）
# 基本参数配置
apiUrl = 'http://apis.juhe.cn/simpleWeather/query'  # 接口请求URL
juheAppKey = os.getenv('juheAppKey')  # 在个人中心->我的数据,接口名称上方查看

def get_weather(city:str):
    # 接口请求入参配置
    requestParams = {
        'key': juheAppKey,
        'city': city,
    }
  # 发起接口网络请求
    response = requests.get(apiUrl, params=requestParams)

    # 解析响应结果
    if response.status_code == 200:
        responseResult = response.json()
        # 网络请求成功。可依据业务逻辑和接口文档说明自行处理。
        print(responseResult)
        return responseResult
    else:
        # 网络异常等因素，解析结果异常。可依据业务逻辑自行处理。
        return '请求异常'

In [5]:
#定义 获取天气的 工具

class GetWeatherInput(BaseModel):
    city: str = Field(description="城市名称")

class GetWeather(BaseTool):
    name: str = Field(default="get_weather")
    description: str = Field(default="这是一个获取天气的API，可以获取指定城市的天气情况，需要用户提供城市名称")
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, city: str) -> str:
        print(f'开始查询{city}的天气信息')
        return get_weather(city)
    async def _arun(self, city: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [7]:
#定义时间工具

import datetime

class GetDateInput(BaseModel):
    format: str = Field(default="%Y-%m-%d", description="日期格式")

class GetDate(BaseTool):
    name: str = Field(default="get_date")
    description: str = Field(default="这是一个获取日期的API，可以获取当前日期")
    args_schema: Type[BaseModel] = GetDateInput

    def _run(self, format: str) -> str:
        return datetime.datetime.now().strftime('%Y-%m-%d')
    async def _arun(self) -> str:
        raise NotImplementedError("This tool does not support async")


In [8]:
# 工具注册
tools = [GetWeather(), GetDate()]
tools

[GetWeather(), GetDate()]

In [9]:
'''
STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION：支持多个输入参数的字符串工具
ZeroShotAgent属于零样本学习型Agent，默认仅支持​​单输入参数的字符串工具​​，并且需要提供工具的描述信息。
CONVERSATIONAL_REACT_DESCRIPTION
'''



#定义agent
agent = initialize_agent(
    tools,
    online_llm,
    AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
    memory=memory, #记忆模块
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_41751/1714861103.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [10]:
#测试调用
result = agent.run("请帮我查询一下北京的天气")
result

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_41751/2129618717.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run("请帮我查询一下北京的天气")




> Entering new AgentExecutor chain...
```json
{
    "action": "get_weather",
    "action_input": "北京"
}
```开始查询北京的天气信息
{'reason': '查询成功!', 'result': {'city': '北京', 'realtime': {'temperature': '22', 'humidity': '60', 'info': '多云', 'wid': '01', 'direct': '东南风', 'power': '1级', 'aqi': '57'}, 'future': [{'date': '2025-05-27', 'temperature': '18/32℃', 'weather': '多云转晴', 'wid': {'day': '01', 'night': '00'}, 'direct': '西南风'}, {'date': '2025-05-28', 'temperature': '19/33℃', 'weather': '晴', 'wid': {'day': '00', 'night': '00'}, 'direct': '西南风'}, {'date': '2025-05-29', 'temperature': '22/33℃', 'weather': '晴', 'wid': {'day': '00', 'night': '00'}, 'direct': '西南风转南风'}, {'date': '2025-05-30', 'temperature': '24/33℃', 'weather': '多云', 'wid': {'day': '01', 'night': '01'}, 'direct': '西南风转南风'}, {'date': '2025-05-31', 'temperature': '22/28℃', 'weather': '阴', 'wid': {'day': '02', 'night': '02'}, 'direct': '东北风'}]}, 'error_code': 0}

Observation: {'reason': '查询成功!', 'result': {'city': '北京', 'realtime': {'t

'北京当前的天气为多云，温度为22℃，湿度为60%，风向为东南风，风力等级为1级，空气质量指数为57。未来几天的天气预报显示，天气将以晴天和多云为主，温度在18至33℃之间波动。'

In [11]:
result = agent.run("今日时间")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "get_date",
    "action_input": ""
}
```
Observation: 2025-05-27
Thought:```json
{
    "action": "Final Answer",
    "action_input": "今天是2025年5月27日。"
}
```

> Finished chain.


'今天是2025年5月27日。'

In [12]:
result = agent.run("我第一次查看了哪个城市的天气")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "您第一次查看了北京的天气。"
}
```

> Finished chain.


'您第一次查看了北京的天气。'

In [16]:
result = agent.run("上一个问题是什么")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "上一个问题是关于推荐北京旅游景点的问题。"
}
```

> Finished chain.


'上一个问题是关于推荐北京旅游景点的问题。'

In [17]:
result = agent.run("去这里旅游，有什么景点推荐")
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "北京有许多著名的旅游景点，例如故宫、天安门广场、长城（如八达岭或慕田峪段）、颐和园、天坛公园、奥林匹克公园以及798艺术区等。此外，您还可以体验胡同文化，品尝地道的北京美食，比如烤鸭和炸酱面。"
}
```

> Finished chain.


'北京有许多著名的旅游景点，例如故宫、天安门广场、长城（如八达岭或慕田峪段）、颐和园、天坛公园、奥林匹克公园以及798艺术区等。此外，您还可以体验胡同文化，品尝地道的北京美食，比如烤鸭和炸酱面。'

In [18]:
#查看agent的memory
agent.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='请帮我查询一下北京的天气', additional_kwargs={}, response_metadata={}), AIMessage(content='北京当前的天气为多云，温度为22℃，湿度为60%，风向为东南风，风力等级为1级，空气质量指数为57。未来几天的天气预报显示，天气将以晴天和多云为主，温度在18至33℃之间波动。', additional_kwargs={}, response_metadata={}), HumanMessage(content='今日时间', additional_kwargs={}, response_metadata={}), AIMessage(content='今天是2025年5月27日。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我第一次查看了哪个城市的天气', additional_kwargs={}, response_metadata={}), AIMessage(content='您第一次查看了北京的天气。', additional_kwargs={}, response_metadata={}), HumanMessage(content='去这里旅游有什么景点推荐', additional_kwargs={}, response_metadata={}), AIMessage(content='请您告诉我具体是哪个城市，我可以为您推荐该城市的旅游景点！', additional_kwargs={}, response_metadata={}), HumanMessage(content='去这里旅游有什么景点推荐', additional_kwargs={}, response_metadata={}), AIMessage(content='请您告诉我具体是哪个城市，我可以为您推荐该城市的旅游景点！', additional_kwargs={}, response_metadata={}), HumanMessage(content='上一个问题是什么', additional_kwargs=